## Getting and Preparing the Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8")
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
data = pd.read_csv("DNN_data.csv", parse_dates = ["time"], index_col = "time")

In [ ]:
data 

In [ ]:
data.info()

In [ ]:
symbol = data.columns[0]
symbol

In [ ]:
data.plot(figsize = (12, 8))
plt.show()

In [ ]:
data["returns"] = np.log(data[symbol] / data[symbol].shift())

In [ ]:
data

## Adding Label/Features

In [ ]:
window = 50

In [ ]:
df = data.copy()
df["dir"] = np.where(df["returns"] > 0, 1, 0)
df["sma"] = df[symbol].rolling(window).mean() - df[symbol].rolling(150).mean()
df["boll"] = (df[symbol] - df[symbol].rolling(window).mean()) / df[symbol].rolling(window).std()
df["min"] = df[symbol].rolling(window).min() / df[symbol] - 1
df["max"] = df[symbol].rolling(window).max() / df[symbol] - 1
df["mom"] = df["returns"].rolling(3).mean()
df["vol"] = df["returns"].rolling(window).std()
df.dropna(inplace = True)

In [ ]:
df

## Adding Feature lags

In [ ]:
lags = 5

In [ ]:
cols = []
features = ["dir", "sma", "boll", "min", "max", "mom", "vol"]

In [ ]:
for f in features:
        for lag in range(1, lags + 1):
            col = "{}_lag_{}".format(f, lag)
            df[col] = df[f].shift(lag)
            cols.append(col)
df.dropna(inplace = True)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
len(cols)

## Splitting into Train and Test Set

In [ ]:
len(df)

In [ ]:
split = int(len(df)*0.66)
split

In [ ]:
train = df.iloc[:split].copy()
train

In [ ]:
test = df.iloc[split:].copy()
test

## Feature Scaling (Standardization)

In [ ]:
train[cols]

In [ ]:
mu, std = train.mean(), train.std() # train set parameters (mu, std) for standardization

In [ ]:
std

In [ ]:
train_s = (train - mu) / std # standardization of train set features

In [ ]:
train_s

In [ ]:
train_s.describe()

## Creating and Fitting the DNN Model

In [ ]:
from DNNModel import *

In [ ]:
# fitting a DNN model with 3 Hidden Layers (50 nodes each) and dropout regularization

set_seeds(100)
model = create_model(hl = 3, hu = 50, dropout = True, input_dim = len(cols))
model.fit(x = train_s[cols], y = train["dir"], epochs = 50, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(train))

In [ ]:
model.evaluate(train_s[cols], train["dir"]) # evaluate the fit on the train set

In [ ]:
pred = model.predict(train_s[cols]) # prediction (probabilities)
pred

In [ ]:
plt.hist(pred, bins = 50)
plt.show()

## Out-Sample Prediction and Forward Testing

In [ ]:
test

In [ ]:
test_s = (test - mu) / std # standardization of test set features (with train set parameters!!!)

In [ ]:
model.evaluate(test_s[cols], test["dir"])

In [ ]:
pred = model.predict(test_s[cols])
pred

In [ ]:
plt.hist(pred, bins = 50);

In [ ]:
test["proba"] = model.predict(test_s[cols])

In [ ]:
test["position"] = np.where(test.proba < 0.47, -1, np.nan) # 1. short where proba < 0.47

In [ ]:
test["position"] = np.where(test.proba > 0.53, 1, test.position) # 2. long where proba > 0.53

In [ ]:
test.index = test.index.tz_localize("UTC")
test["NYTime"] = test.index.tz_convert("America/New_York")
test["hour"] = test.NYTime.dt.hour

In [ ]:
test["position"] = np.where(~test.hour.between(2, 12), 0, test.position) # 3. neutral in non-busy hours

In [ ]:
test["position"] = test.position.ffill().fillna(0) # 4. in all other cases: hold position

In [ ]:
test.position.value_counts(dropna = False)

In [ ]:
test["strategy"] = test["position"] * test["returns"]

In [ ]:
test["creturns"] = test["returns"].cumsum().apply(np.exp)
test["cstrategy"] = test["strategy"].cumsum().apply(np.exp)

In [ ]:
test[["creturns", "cstrategy"]].plot(figsize = (12, 8))
plt.show()

In [ ]:
ptc = 0.000059

In [ ]:
test["trades"] = test.position.diff().abs()

In [ ]:
test.trades.value_counts()

In [ ]:
test["strategy_net"] = test.strategy - test.trades * ptc

In [ ]:
test["cstrategy_net"] = test["strategy_net"].cumsum().apply(np.exp)

In [ ]:
test[["creturns", "cstrategy", "cstrategy_net"]].plot(figsize = (12, 8))
plt.show()